In [ ]:
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [ ]:
path = "../data/Receptanteckningar_justerad.xlsx"
df = pd.read_excel(path, sheet_name=None)

### 'Ej AB 2018' is the sheet where no antibiotics where prescribed.

In [ ]:
no_antibiotics = df['Ej AB 2018']
with pd.option_context('display.max_colwidth', None):
    display(no_antibiotics.head(5))

### Removing all nan cells

In [ ]:
nan_remove = np.where(pd.isnull(no_antibiotics))[0].tolist()
no_antibiotics = no_antibiotics.drop(nan_remove).reset_index(drop=True)

In [ ]:
no_antibiotics.tail()

In [ ]:
no_antibiotics.shape

### Removing all cells starting with Elektroniskt recept 
This is since they only contain medicine prescribed and we supposed that was written after patients visits

In [ ]:
no_antibiotics['Anteckning'][52]

In [ ]:
def remove_recipe_rows(dataframe):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains('Elektroniskt recept', regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_rows = l*idx
    remove_rows = remove_rows[remove_rows!=0]
    remove_rows = remove_rows[~np.isnan(remove_rows)]
    return remove_rows

In [ ]:
remove_rows = remove_recipe_rows(no_antibiotics)
no_antibiotics_new = no_antibiotics.drop(remove_rows).reset_index(drop=True)
with pd.option_context('display.max_colwidth', None):
    display(no_antibiotics_new.head(10))

In [ ]:
no_antibiotics_new.shape

### Removing html tags
Inspecting the data, we noticed that some notes contained html tags, as seen in journal note 0 above.   
We can use the web scraping framework Beautiful Soup to return the text inside the tags

In [ ]:
def remove_html_tags(dataframe):
    for i,t in enumerate(dataframe['Anteckning']):
        ct = BeautifulSoup(t, "lxml").text
        dataframe['Anteckning'][i] = ct

In [ ]:
with pd.option_context('display.max_colwidth', None):
    remove_html_tags(no_antibiotics_new)

In [ ]:
no_antibiotics_new['Anteckning'][0]

### Masking the years shown in text
Year should not be a feature we want the model to learn.   
We want the model to learn based on the symptoms the patient has.   
Therefore, we remove the years  

Some years are written as 20xx others as just xx

In [ ]:
def remove_year(dataframe, year):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains(year, regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_y = l*idx
    remove_y = remove_y[remove_y!=0]
    print(remove_y)    
    for i in remove_y:
        dataframe['Anteckning'][i] = re.sub(year, 'Year-', dataframe['Anteckning'][i])

In [ ]:
for i in range(2011,2021,1):
    year = str(i)+'-'
    remove_year(no_antibiotics_new, year)

In [ ]:
no_antibiotics_new['Anteckning'][123]

In [ ]:
for i in range(11,21,1):
    year = str(i)+'-'
    remove_year(no_antibiotics_new, year)    

In [ ]:
no_antibiotics_new['Anteckning'][123]

In [ ]:
for i in range(11,21,1):
    year = str(i)+'/'
    remove_year(no_antibiotics_new, year)    

In [ ]:
no_antibiotics_new['Anteckning'][645]

### Removing antibiotics from text
If the model learns to identify different antibiotics in the text, it will not learn topick up the general symptoms 

In [ ]:
antib = ["kåvepenin", "tikacillin", "avopen", "peceve", "dalacin", "clindamycin", "amimox", 
         "amoxicillin", "imacillin", "doxyferm", "doxycyklin", "vibranord", "abboticin", 
         "ery-max", "flagyl", "metronidazol", "primve", "metronidazol", "klindamycin",
        "Kåvepenin", "Tikacillin", "Avopen", "Peceve", "Dalacin", "Clindamycin", "Amimox", 
         "Amoxicillin", "Imacillin", "Doxyferm", "Doxycyklin", "Vibranord", "Abboticin", 
         "Ery-max", "Flagyl", "Metronidazol", "Primve", "Metronidazol", "Llindamycin", 
        "pc", "penicillin", "antibiotika", "ab", "Pc", "Penicillin", "Antibiotika", "Ab", 
        "PC", "AB"]

def remove_drug(dataframe, drug):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains(drug, regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_y = l*idx
    remove_y = remove_y[remove_y!=0]
    print(len(remove_y), remove_y)   
    for i in remove_y:
        dataframe['Anteckning'][i] = re.sub(drug, ' ', dataframe['Anteckning'][i])

In [ ]:
# Example of where `kåvepenin` and `citodon` is mentioned
no_antibiotics_new['Anteckning'][3737]

In [ ]:
for a in antib:
    remove_drug(no_antibiotics_new, a)

In [ ]:
# antibiotics is now removed
no_antibiotics_new['Anteckning'][3737]

### Removing other keywords
Other commonly occuring words in the text are now removed, such as "`Osignerade anteckningar`"   
Note, that we can see names of dentists and the patient, which **VERY bad!**
We, therefore, also apply NER to annonymize the data later

In [ ]:
no_antibiotics_new['Anteckning'][2484]

In [ ]:
def remove_osi(dataframe):
    """Removes 'Osignerade anteckningar' from notes."""
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains('Osignerade', regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_rows = l*idx
    remove_rows = remove_rows[remove_rows!=0]
    remove_rows = remove_rows[~np.isnan(remove_rows)]
    print(remove_rows[:10])
    return remove_rows

In [ ]:
remove_rows = remove_osi(no_antibiotics_new)
no_antibiotics_new = no_antibiotics_new.drop(remove_rows).reset_index(drop=True)
with pd.option_context('display.max_colwidth', None):
    display(no_antibiotics_new.head(5))

In [ ]:
no_antibiotics_new['Anteckning'][2484]

In [ ]:
no_antibiotics_new['Anteckning'][38]

In [ ]:
no_antibiotics_new['Anteckning'][123]

In [ ]:
def remove_srn(dataframe):
    """Removes 'Skriver recept' from notes."""
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains('Skriver recept', regex=True)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_rows = l*idx
    remove_rows = remove_rows[remove_rows!=0]
    remove_rows = remove_rows[~np.isnan(remove_rows)]
    print(remove_rows[:10])
    print(len(remove_rows))
    return remove_rows

In [ ]:
remove_rows = remove_srn(no_antibiotics_new)
for i in remove_rows:
    sentences = no_antibiotics_new['Anteckning'][i].split('.')
    for s in sentences:
        if 'Skriver recept' in s:
            sentences.remove(s)
    no_antibiotics_new['Anteckning'][i] = ".".join(sentences)

In [ ]:
no_antibiotics_new['Anteckning'][38]

In [ ]:
no_antibiotics_new['Anteckning'][123]

In [ ]:
no_antibiotics_new['Anteckning'][1547]

In [ ]:
def remove_empty(dataframe):
    l = []
    for i,text in enumerate(dataframe['Anteckning']):
        if len(text) == 0:
            l.append(i)
    print(l[:10])
    print(len(l))
    return l 

In [ ]:
remove_rows = remove_empty(no_antibiotics_new)
no_antibiotics_new = no_antibiotics_new.drop(remove_rows).reset_index(drop=True)

In [ ]:
no_antibiotics_new['Anteckning'][1547]

## Grouping cells by `unikt nummer`, `datum` and `klinik`

In [ ]:
with pd.option_context('display.max_colwidth', None):
    merge_no_antibiotics = no_antibiotics_new.groupby(['Unikt nummer','Datum','Klinik'])['Anteckning'].apply(lambda Anteckning: ''.join(Anteckning.to_string(index=False))).str.replace('(\\n)', ' ').reset_index()
    display(merge_no_antibiotics.head(15))

### Remove spaces

In [ ]:
merge_no_antibiotics['Anteckning'][0]

In [ ]:
l = merge_no_antibiotics.shape[0]
for i in range(l):
    line = merge_no_antibiotics['Anteckning'][i]
    merge_no_antibiotics.at[i, 'Anteckning'] = " ".join(line.split())

In [ ]:
merge_no_antibiotics['Anteckning'][0]

In [ ]:
l = merge_no_antibiotics.shape[0]
for i in range(l):
    line = merge_no_antibiotics['Anteckning'][i]
    merge_no_antibiotics.at[i, 'Anteckning'] = line.lower()

In [ ]:
merge_no_antibiotics['Anteckning'][0]

In [ ]:
with pd.option_context('display.max_colwidth', None):
    merge_no_antibiotics.to_csv('Ej_AB.csv')

## Getting statistics of visits per patient

In [ ]:
a = merge_no_antibiotics['Unikt nummer'].value_counts(sort=True).values
unique, counts = np.unique(a, return_counts=True)
l = len(a)
D = dict(zip(unique, counts/l))
for k,v in D.items():
    print(k,v)

In [ ]:
plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
plt.show()

In [ ]:
merge_no_antibiotics.shape

# Repeat all cleaning steps for all years in the `AB` spreadsheet 
'AB' spread sheets are from antibiotics prescriptions

In [ ]:
antibiotics = pd.concat([df['AB 2012'], df['AB 2014'], df['AB 2017'], df['AB 2018']], axis=0).reset_index(drop=True)

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(antibiotics.head(20))

In [ ]:
antibiotics.shape

### Removing nan from cells

In [ ]:
nan_remove = np.where(pd.isnull(antibiotics))[0].tolist()
antibiotics = antibiotics.drop(nan_remove).reset_index(drop=True)

In [ ]:
np.where(pd.isnull(antibiotics))[0]

### Removing all cells starting with Elektroniskt recept since they only contain medicine prescribed and we supposed that was written after patients visits

In [ ]:
remove_rows = remove_recipe_rows(antibiotics)
antibiotics_new = antibiotics.drop(remove_rows).reset_index(drop=True)
with pd.option_context('display.max_colwidth', None):
    display(antibiotics_new.tail(10))

In [ ]:
antibiotics_new.shape

In [ ]:
antibiotics_new['Anteckning'][24013]

### Removing html tags

In [ ]:
with pd.option_context('display.max_colwidth', None):
    remove_html_tags(antibiotics_new)

In [ ]:
antibiotics_new['Anteckning'][24013]

### Masking years

In [ ]:
for i in range(2011,2021,1):
    year = str(i)+'-'
    remove_year(antibiotics_new, year)

In [ ]:
for i in range(11,21,1):
    year = str(i)+'-'
    remove_year(antibiotics_new, year)    

In [ ]:
for i in range(2011,2021,1):
    year = str(i)+'/'
    remove_year(antibiotics_new, year)

In [ ]:
for i in range(11,21,1):
    year = str(i)+'/'
    remove_year(antibiotics_new, year)    

In [ ]:
antibiotics_new['Anteckning'][7]

### Removing antibiotics

In [ ]:
for a in antib:
    remove_drug(antibiotics_new, a)

In [ ]:
antibiotics_new['Anteckning'][7]

In [ ]:
antibiotics_new['Anteckning'][2]

In [ ]:
remove_rows = remove_osi(antibiotics_new)
antibiotics_new = antibiotics_new.drop(remove_rows).reset_index(drop=True)
with pd.option_context('display.max_colwidth', None):
    display(no_antibiotics_new.head(10))

In [ ]:
antibiotics_new['Anteckning'][2]

In [ ]:
antibiotics_new['Anteckning'][1652]

In [ ]:
antibiotics_new['Anteckning'][1850]

In [ ]:
def remove_srn(dataframe):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains('Skriver recept', regex=True)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_rows = l*idx
    remove_rows = remove_rows[remove_rows!=0]
    remove_rows = remove_rows[~np.isnan(remove_rows)]
    #new = df.drop(remove_rows)
    print(remove_rows[:10])
    print(len(remove_rows))
    return remove_rows

In [ ]:
remove_rows = remove_srn(antibiotics_new)
for i in remove_rows:
    sentences = antibiotics_new['Anteckning'][i].split('.')
    for s in sentences:
        if 'Skriver recept' in s:
            sentences.remove(s)
    antibiotics_new['Anteckning'][i] = ".".join(sentences)

In [ ]:
antibiotics_new['Anteckning'][1652]

In [ ]:
remove_rows = remove_empty(antibiotics_new)
antibiotics_new = antibiotics_new.drop(remove_rows).reset_index(drop=True)

### Groupping by dates

In [ ]:
with pd.option_context('display.max_colwidth', None):
    merge_antibiotics = antibiotics_new.groupby(['Unikt nummer','Datum','Klinik'])['Anteckning'].apply(lambda Anteckning: ''.join(Anteckning.to_string(index=False))).str.replace('(\\n)', ' ').reset_index()
    display(merge_antibiotics.head(10))

## Statistics from patients visits

In [ ]:
a = merge_antibiotics['Unikt nummer'].value_counts(sort=True).values
idx = merge_antibiotics['Unikt nummer'].value_counts(sort=True).index
unique, counts = np.unique(a, return_counts=True)
l = len(a)
D = dict(zip(unique, counts/l))
for k,v in D.items():
    print(k,v)

In [ ]:
plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
plt.show()

In [ ]:
mask = np.where(a>1,True,False)

In [ ]:
visits = np.array([mask*idx])
visits = visits[visits!=0]
sep = []
for v in visits:
    dates = merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == v].values
    for i in range(1,len(dates)):
        days = dates[i] - dates[i-1]
        sep.append(int(days.astype('timedelta64[D]')/ np.timedelta64(1, 'D')))

In [ ]:
sep = np.array(sep)
month = sep[sep<30]
print('Percentage of second visits within 30 days', len(month)/len(sep))

unique, counts = np.unique(month, return_counts=True)
l = len(month)
D = dict(zip(unique, counts/l))
for k,v in D.items():
    print(k,v)

In [ ]:
plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
plt.show()

In [ ]:
#merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == 98].values
#merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == 98].index
with pd.option_context('display.max_colwidth', None):
    display(merge_antibiotics.iloc[380:386])

In [ ]:
visits = np.array([mask*idx])
visits = visits[visits!=0]
for v in visits:
    dates = merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == v].values
    for i in range(1,len(dates)):
        days = dates[i] - dates[i-1]
        days = int(days.astype('timedelta64[D]')/ np.timedelta64(1, 'D'))
        if days<15:
            ind = merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == v].index[i]
            merge_antibiotics.at[ind, 'Datum'] = dates[i-1]
            dates[i] = dates[i-1]

In [ ]:
merge_antibiotics['Datum'][merge_antibiotics['Unikt nummer'] == 98].values

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(merge_antibiotics.iloc[438:449])

In [ ]:
with pd.option_context('display.max_colwidth', None):
    merge_antibiotics = merge_antibiotics.groupby(['Unikt nummer','Datum','Klinik'])['Anteckning'].apply(lambda Anteckning: ''.join(Anteckning.to_string(index=False))).str.replace('(\\n)', ' ').reset_index()
    display(merge_antibiotics.head(10))


### Remove spaces

In [ ]:
merge_antibiotics['Anteckning'][6]

In [ ]:
l = merge_antibiotics.shape[0]
for i in range(l):
    line = merge_antibiotics['Anteckning'][i]
    merge_antibiotics.at[i, 'Anteckning'] = " ".join(line.split())

In [ ]:
merge_antibiotics['Anteckning'][6]

In [ ]:
l = merge_antibiotics.shape[0]
for i in range(l):
    line = merge_antibiotics['Anteckning'][i]
    merge_antibiotics.at[i, 'Anteckning'] = line.lower()

In [ ]:
merge_no_antibiotics['Anteckning'][6]

# Plot number of character in journal notes
The distribution of the different journal entries (when patient was and wasn't prescribed antibiotics) are compares and plotted   
The goal is for dentists to take the same number of notes or more when prescribing antobiotics and there should always be as "sufficiently" long journal notes.  

Also for explainability to be extracted directly from the tranied Transformer models (s.a. BERT) it is benificial if the number of words are 512 or fewer.   
In English, the average word lenght ~4 characters

In [ ]:
l = merge_antibiotics.shape[0]
length = []
maxn = 0
minn = 1000
idx_max = 0
idx_min = 0

for i in range(l):
    line = merge_antibiotics['Anteckning'][i]
    length.append(len(line))
    if len(line)>maxn:
        maxn = len(line)
        idx_max = i
    if len(line)<minn:
        minn = len(line)
        idx_min = i
        
    
print(np.mean(length))
print(min(length))
print(max(length))

print(maxn, minn)
print(idx_max, idx_min)

sns.distplot(length)
plt.xlabel('Characters count antibiotics');

In [ ]:
merge_antibiotics['Anteckning'][488]

In [ ]:
merge_antibiotics['Anteckning'][4194]

In [ ]:
l = merge_no_antibiotics.shape[0]
length = []
maxn = 0
minn = 1000
idx_max = 0
idx_min = 0
for i in range(l):
    line = merge_no_antibiotics['Anteckning'][i]
    length.append(len(line))
    if len(line)>maxn:
        maxn = len(line)
        idx_max = i
    if len(line)<minn:
        minn = len(line)
        idx_min = i
        
    
print(np.mean(length))
print(min(length))
print(max(length))

print(maxn, minn)
print(idx_max, idx_min)

sns.distplot(length)
plt.xlabel('Char count no antibiotics');

In [ ]:
merge_antibiotics['Anteckning'][3838]

In [ ]:
merge_antibiotics['Anteckning'][2568]

## Store the cleaned notes for Antibiotics

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(merge_antibiotics.tail())
    merge_antibiotics.to_csv('AB.csv')

In [ ]:
with pd.option_context('display.max_colwidth', None):
    dataset={'Anteckning':[m for m in merge_antibiotics['Anteckning']],
        'Label': [1 for i in range(len(merge_antibiotics['Anteckning']))]}

In [ ]:
with pd.option_context('display.max_colwidth', None):
    for n in merge_no_antibiotics['Anteckning']:
        dataset['Anteckning'].append(n)
        dataset['Label'].append(0)

In [ ]:
dataset_antibiotics = pd.DataFrame(dataset, columns = ['Anteckning', 'Label'])
dataset_antibiotics.head()

In [ ]:
remove_rows = remove_empty(dataset_antibiotics)
dataset_antibiotics = dataset_antibiotics.drop(remove_rows).reset_index(drop=True)

## Remove Names
The name were extracted using KB BERT NER. The list of names were then used here to remove first and last names and certain abreviations, s.a. "FTV" 

In [ ]:
def remove_name(dataframe,name):
    df1 = dataframe['Anteckning'].iloc[:]
    bool_list = df1.str.contains(name, regex=False)
    idx = bool_list.tolist()
    l = np.arange(len(idx))
    remove_y = l*idx
    remove_y = remove_y[remove_y!=0]
    for i in remove_y:
        dataframe['Anteckning'][i] = re.sub(name + ' ', ' ', dataframe['Anteckning'][i])

    return remove_rows

In [ ]:
names = ['Klar', 'Käkkir', 'Cecilia', 'Wexell', 'Neuront', 'Lisa', 'Hårsmar', 'FTV', 'Fixop', 'Straumann', 'Alexandra', 'Gunilla', 'Laine', 'Västrik', 'IR', 'Hannes', 'Olsson', 'Tissueve', 'Reet', 'Karlsson', 'Diagn', 'TissueLeve', 'Ak', 'Patolan', 'regi', 'Kaoru', 'Oyaiz', 'BoneLevel', 'Sepcaine', 'Anamn', 'Akut', 'Bengt', 'Peririn', 'Helstatus', 'Tel', 'Stefan', 'Td', 'Ahmed', 'Marianne', 'Flor', 'Vahed', 'Christina', 'Backe', 'Anamnes', 'Ketac', 'Molar', 'Molarsalilo', 'Yeareo', 'Temp', 'Bond', 'Sofia', 'konstant', 'Mari', 'Louise', 'Odersjö', 'Helmbäck', 'Boda', 'Adam', 'Hjäl', 'AKUT', 'Dak', 'manuell', 'Andole', 'Na', 'Leaf', 'Oral', 'Artling', 'Martin', 'Mvh', 'Tandl', 'Malin', 'Tina', 'Thakhrar', 'Alice', 'Signe', 'AS', 'Per', 'HB', 'Dag', 'RN', 'SLA', 'Sens', 'Pia', 'Diana', 'Har', 'Göran', 'Kjeller', 'Johan', 'Malmström', 'Digano', 'Ketacfil', 'Pol', 'Ip', 'Anna', 'Lena', 'Citod', 'Nybesök', 'Kerstin', 'Persson', 'Pamolei', 'Pamol', 'Trado', 'Mattias', 'Anv', 'Hexiden', 'Dal', 'Wolak', 'Folkesson', 'Chlumsky', 'Katrin', 'Raunosal', 'LE', 'bro', 'ÅB', 'Remitte', 'Jenny', 'Petersson', 'Vahedi', 'Fort', 'Ipren', 'Ny', 'Kerstn', 'Cav', 'Harriet', 'Birgitta', 'dr', 'Leif', 'Lind', 'Ring', 'Madelene', 'Sanjiv', 'Kanagaraja', 'Op', 'Aron', 'Ascher', 'Yvonne', 'Anestesi', 'Id', 'MW', 'Björlins', 'Kir', 'Ling', 'Shiva', 'Raoufi', 'Carina', 'Vegard', 'Garsjö', 'Kurbanali', 'Alimi', 'Sylte', 'Kron', 'NaC', 'Corsody', 'Perk', 'Catharina', 'Jacobsson', 'Pedodonti', 'Hossein', 'Kashani', 'Sanji', 'Kanagaraj', 'Larsson', 'Nina', 'Annika', 'Piez', 'He', 'Alexander', 'Dikran', 'Nellie', 'JIA', 'Jan', 'Erik', 'Johansson', 'IK', 'Par', 'Nikos', 'Merakos', 'Paroe', 'KAR', 'Fogelin', 'Maria', 'Remissvar', 'Elis', 'Kvissberg', 'Peter', 'Kofron', 'Or', 'PU', 'Akutus', 'Amalgam', 'Machi', 'Papaioannou', 'Dr', 'Holger', 'Ann', 'KS', 'Saer', 'Amir', 'Rehrmann', 'Peri', 'ANAMNES', 'Allm', 'Jacobsen', 'Marie', 'Colt', 'Lisbeth', 'Svalander', 'Christian', 'Schaefer', 'Hu', 'Fried', 'Eriksson', 'Glen', 'Levander', 'Bone', 'Level', 'Bråne', 'Del', 'Lesnukhin', 'Temoda', 'Kristina', 'Goldkuhl', 'Daganteck', 'Flors', 'St', 'Hill', 'Ramipril', 'Amlodipin', 'ML', 'Wahlqvist', 'Kodein', 'Trombykl', 'Dolconti', 'Eva', 'Wridell', 'Tiseve', 'Neck', '46', 'TissueLevel', 'Wide', 'SL', 'Alveo', 'Heikki', 'War', 'Lise', 'Lotte', 'Löfqvist', 'Protetik', 'Ast', 'Bru', 'Guldheden', 'TVR', 'Ped', 'tsk', 'Lok', 'Xyl', 'Anamnmg', 'Patt', 'Akoto', 'Orudi', 'Klara', 'Alvend', 'Madeleine', 'Dakote', 'Smile', 'Vb500mg', 'Öhrnell', 'mr', 'Ink', 'Anne', 'Christine', 'Cederström', 'Ja', 'Leo', 'Victor', 'Bastante', 'End', 'Emdogain', 'LO', 'Prolén', 'Listeri', 'Nozininet', 'Allayardy', 'eth', 'Cavalli', 'Björkman', 'Heracillin', 'Inger', 'Mona', 'Mikati', 'An', 'Kenny', 'Andersson', 'Beh', 'Waltbrand', 'Reto', 'Walterbrand', 'Janson', 'Ulrika', 'Gunnarsson', 'Alvedon', 'Patoka', 'Lars', 'Olof', 'Avesta', 'Haider', 'Alveol', 'lott', 'Remiss', 'Rosengren', 'KLAR', 'Manuel', 'GS', 'Örnell', 'David', 'Öhman', 'Mats', 'Wallström', 'Eliqu', 'Roger', 'Akutu', 'DOola', 'Citodon', 'Lil', 'Dakins', 'Zekry', 'Ery', 'Max', 'Mo', 'Öhrnellkir', 'Kåvepen', 'Dennis', 'Ap', 'Pal', 'helger', 'Patrik', 'Vladimir', 'Susanne', 'Piezo', 'Sellgren', 'Midazola', 'NacL', 'Oss', 'Gide', 'Standard', 'Ne', 'Endo', 'Spec', 'DO', 'John', 'Bratel', 'Ftv', 'Kulan', 'Von', 'Essen', 'Ulf', 'Cristnitez', 'Bon', 'Rens', 'Rit', 's', 'Lundh', 'Vidaza', 'Hege', 'Garelius', 'Westin', 'Sten', 'Mellqvist', 'Leszek', 'Zomet', 'Xgeos', 'Ur', 'Ingela', 'Lissbrant', 'Hem', 'Pitack', 'Örtengren', 'Mel', 'Anki', 'Ceram', 'Mono', 'Maxillarsin', 'Uts', 'Nadja', 'Berit', 'BW', 'Kristoffer', 'Rehr', 'Mattis', 'Josephine', 'Calwell', 'Luc', 'Åsa', 'Brånemark', 'Ti', 'Unite', 'Kanagarajapn', 'Örnhall', 'Britse', 'Tomas', 'Nelsén', 'Boka', 'Adper', 'Scotchbon', 'Filtek', 'Alberts', 'Albert', 'Morgan', 'Giovanni', 'Serino', 'Vikrylcel', 'Li', 'Lilliehöök', 'TH', 'Cristnit', 'Ivanoff', 'Cinacef', 'Rec', 'Vicryl', 'Linda', 'Spoln', 'NaOC', 'Annelie', 'Amanda', 'Klas', 'Ulenius', 'Akutunn', 'Dakinsoso', 'Diff', 'Eija', 'Gita', 'Gal', 'Blom', 'Karin', 'Camilla', 'Aneste', 'Björn', 'Fürst', 'Top', 'Monica', 'Mb', 'Fil', 'Supreme', 'Sup', 'Tromby', 'Rot', 'Widman', 'Fer', 'Tradi', 'Panodil', 'Michelle', 'Ylva', 'von', 'Zweigbergk', 'Askim', 'Den', 'Clinomyn', 'Zeki', 'Yarbsal', 'Yarbug', 'AK', 'Corsodyl', 'Mir', 'Orudimg', 'Fuji', 'II', 'Nils', 'Buck1', 'Lokalanes', 'Sjöberg', 'Andreas', 'Schultz', 'Perkömmn', 'Rapid', 'Åker', 'vari', 'Sunita', 'VP', 'Petra', 'Lindh', 'Marita', 'Thor', 'Malekzadeh', 'Karies', 'Row', 'Lindemanns', 'Rythén', 'Johannes', 'Paulsson', 'Wass', 'Återbesök', 'Zo', 'Nobetec', 'Eyal', 'Muale', 'ip', 'Dalin', 'el', 'Britt', 'Sofie', 'Hansén', 'och', 'Lipina', 'Samma', 'Elliot', 'Kävepen', 'Calasep', 'Zoe', 'Gun', 'Lindquist', 'Anja', 'Claudia', 'Silva', 'Groz', 'Inj', 'Dimitrios', 'Karazisis', 'Gunnar', 'Stridh', 'Ali', 'hel', 'Dahlström', 'Strömvall', 'Nasaluber', 'KL', 'Jessica', 'Svanevik', 'Duotrav', 'Esidre', 'Metoprolo', 'Orion', 'Tdl', 'Stina', 'Malm', 'Rhemans', 'Lindome', 'Tommy', 'Saia', 'Vainalis', 'Tore', 'Däldborg', 'Ing', 'Jonasson', 'Pus', 'Razan', 'Dagck', 'Rödbo', 'Akutper', 'Sylvia', 'Danuta', 'Crist', 'Niklas', 'Lindgren', 'Alvido', 'Lovisa', 'mari', 'Cassel', 'Cem', 'TTF', 'Arvidsson', 'Cob', 'Cre', 'us', 'Mia', 'Carlsson', 'Pericoron', 'Hanna', 'Sta', 'Lev', 'Kostas', 'Bougas', 'Borås', 'Zoya', 'Holm', 'Sval', 'Morfin', 'Eliquis', 'Remittent', 'Rem', 'Anders', 'Franzén', 'Masoume', 'Zarandi', 'Arijan', 'Ismajl', 'dagl', 'Cit', 'Coltosol', 'Fuiji', 'Tre', 'EDTA', 'Nathalie', 'Sp', 'Jin', 'Patricia', 'Ulla', 'Berglundh', 'Zometa', 'Xgeva', 'Hofgren', 'Ash', 'Amin', 'Selma', 'Jakob', 'Bill', 'LM', 'Sara', 'Brimstedt', 'Roya', 'Farzini', 'Molin', 'Frida', 'Hallgren', 'MeliorC', 'Michailidis', 'Horred', 'Torbjörn', 'Pac', 'Bursell', 'Xge', 'Patogen', 'Ev', 'marg', 'Rinex', 'Patuki', 'Waran', 'Cobolt', 'Crom', 'Collacone', 'Lundby', 'Spol', 'Nacl', 'Radhi', 'Diakins', 'Coesoft', 'AkC', 'Klarutu', 'Wave', 'One', 'Gold', 'Berglundigo', 'Wallström800mg', 'Lennart', 'Ana', 'Menezes', 'Dormicu', 'Therese', 'Michael', 'Valentin', 'Ref', 'Pam', 'Citodonod', 'Helen', 'Lut', 'Olivia', 'Shahnaz', 'Ingrid', 'and', 'Killey', 'Ser', 'GW', 'GB', 'RosMarie', 'ED', 'Axel', 'Lindberg', 'Meda', 'Alex', 'Zmg', 'Willner', 'Dan', 'Andre', 'Thyg', 'JYYea', 'Annaliniken', 'Isa', 'FK', 'Alvdon', 'GÖ', 'Fe', 'Golnar', 'Hjällbo', 'trol', 'Alveolit', 'Ulrike', 'Månstigen', 'Patmn', 'Elins', 'Stat', 'NaCl', 'Vera', 'Asyludi', 'Alkutus', 'Dick', 'Dockelberg', 'ZO', 'Plan', 'fort', 'Ch', 'Dalgren', 'Amixicillin', 'Olemyr', 'Hälsohuset', 'Ulrica', 'In', 'Mirwais', 'Ahmadzai', 'Tu', 'Patpå', 'NaCny', 'Kollega', 'Afrodite', 'KK', 'Hudit', 'Hansson', 'Park', 'Kävepenmg', 'allm', 'Lindkvist', 'Rehrmans', 'Nora', 'Min', 'Topas', 'Volodymyr', 'ANAMN', 'Allmän', 'Alva', 'Emma', 'Kjell', 'Riv', 'Gadler', 'Trombyolovast', 'EDA', 'Börjesson', 'TetricFlow', 'Wallander', 'Krook', 'Orudis', 'Claesson', 'Ammar', 'ID', 'Coltoso', 'Ringkir', 'Dasmah', 'Gum', 'Paroex', 'Gel', 'Patdy', 'Inrem', 'Bah', 'Peiman', 'Excit', 'Flo', 'Ari', 'Samuelsson', 'Kons', 'NÄL', 'Fredrik', 'Widar', 'Tanja', 'Vis', 'Lorena', 'Th', 'AF', 'Genrell', 'Maj', 'Nilsson', 'Evert', 'Migran', 'Db', 'Dental', 'Fl', 'AP', 'Gråbo', 'Parxg', 'volt', 'Bingxi', 'Emina', 'Gvozdarxi', 'Gvozdar', 'Vic', 'Mor', 'Adr', 'Adel', 'Sundblad', 'JonasJohnson', 'ÖNH', 'Nyström', 'Alberto', 'Turri', 'KG', 'Trauma', 'Dahlén', 'Alveolar', 'Quick', 'Magnél', 'Coll', 'Patoe', 'Rönnqvist', 'Hawerman3950', 'Öhnell', 'Wertsén', 'Flytström', 'AA', 'Youse', 'Konstant', 'Pors', 'Arvidssons', 'KKR', 'Alma', 'Fr', 'Ketogan', 'Ketoganod', 'Tramadol', 'Pateri', 'Cordy', 'Olga', 'Jenssen', 'Adina', 'Popa', 'TeP', 'Ang', 'Lundqvist', 'Kristofer', 'Konopelk', 'Cerami', 'Ida', 'Kersin', 'Bo', 'Kinna', 'Tor', 'Widbom', 'Christer', 'Storjour', 'Kajsa', 'Che', 'Sundström', 'Nobote', 'fm', 'CPAform', 'Ewa', 'Eylertsen', 'Elin', 'Ahl', 'Ringe', 'PS', 'Es', 'Nadaief', 'Ibumeti', 'Mira', 'PatC', 'Bro300mg', 'Bron', 'Kari', 'Jäwer', 'Anette', 'MOlar', 'Endont', 'Fredell', 'Mikael', 'Åby', 'ons', 'Linnea', 'Patsody', 'Hultin', 'AT', 'Gin', 'Ada', 'Matthias', 'Går', 'bj', 'Thomas', 'Håkansson', 'Arion', 'Tetric', 'Evo', 'Nordberg', 'Parod', 'Hy', 'San', 'Cyrettage', 'Johanson', 'Sandra', 'Sulfa', 'Ekestubbe', 'Alternova', 'Pietzo', 'Oc', 'Kariesreg', 'KB', 'Elma', 'Misirlic', 'XA', 'Greger', 'Pelle', 'Cli', 'mamm', 'Lalin', 'Ekelund', 'Septocaine', 'Elsa', 'Granulom', 'Siva', 'Nargis', 'Nasiri', 'Domäng', 'fru', 'Esters', 'Vik', 'Novum', 'Kåeni', 'hann', 'Jennifer', 'Ytter', 'Inga', 'Aplas', 'William', 'Persantin', 'Iliev', 'Kil', 'Rignell', 'Nordin', 'Aldell800mg', 'Zarah', 'Jimmy', 'Jour', 'Åter', 'Simon', 'Annes', 'Tandvård', 'Pedodont', 'Perkar', 'Ulvestam', 'Lassing', 'Eliquiz', 'Te', 'Prem', 'Wa', 'Emanuelsson', 'Lamrig', 'Aleksand', 'Lundgren', 'Premed', 'Anman', 'Oraseal', 'Ren', 'Lidoka', 'Lina', 'Patsanti', 'Seloken', 'Od', 'Agneta', 'BoÅ', 'Lie', 'Scotbon', 'EvoCera', 'Tissu', 'Tetyan', 'Tacos', 'Ghazaleh', 'Boroman', 'Volmefjord', 'Nai', 'SP', 'Julia', 'Buck', 'Käk', 'Sofi', 'Brinkemo', 'Borelia', 'Patric', 'Dakod', 'Perma', 'Sharp', 'Ilon', 'Daniel', 'EVA', 'Morit10', 'Nobete', 'Szanto', 'Annelii', 'Malmqvist', 'Yearmg', 'Sandoz', 'Film', 'Anas', 'Undersök', 'Alexan', 'Rapidsutu', 'Delvert', 'Sut', 'Amoxicllin', 'Ind', 'Il', 'Bie', 'Holtz', 'Schondelma', 'Patmgmg', 'Bod', 'Patkir', 'Sixten', 'HJ', 'Brå', 'Ivan', 'Pe', 'len', 'IT', 'Solbjörg', 'NE', 'AAL', 'Ena', 'SE', 'Burgos', 'Dahlgren', 'LEL', 'Widmark', 'Ötd', 'CAVE', 'Bu', 'Glivecoe', 'Ket', 'Bash', 'Ralf', 'Caries', 'Ahmadza', 'Pernilla', 'Angelus', 'Hi', 'Yahya', 'Rodica', 'Dragan', 'Kalcipoloac', 'Imovan', 'Thy', 'Wahlberg', 'Plugger', 'Peg', 'Reinfo', 'Ragnar', 'Hanås', 'JK', 'Patmg', 'Eliques', 'Ringer', 'Dela', 'Strömblad', 'Henrik', 'TEL', 'Zina', 'Kam', 'Östgärd', 'Iso', 'Life', 'Patepraz', 'Major', 'Angel', 'Nikolaos', 'Rigas', 'Carl', 'Stade', 'Patolärutskottetutskottet', 'Kihl', 'Satur', 'Åke', 'Marika', 'Hälso', 'Toss', 'Zneugenol', 'Kåve', 'KSod', 'King', 'Son', 'Balverdi', 'Ipola', 'Romexex', 'Er', 'Kariesbeh', 'Bodil', 'Orbit', 'Dam', 'Lekholm', 'Zarb', 'Angesjö', 'Stoke', 'Folacjen', 'Heredit', 'MK', 'Anamnarelt', 'Syberyjska', 'Mariana', 'Edvinsson', 'Decima', 'Gusmao', 'Amal', 'AkutTV', 'Jim', 'Axelsson', 'Melior', 'Borg', 'Make', 'Ibrahim', 'Tak', 'Hatami', 'rielson', 'Hosssein', 'Ine', 'Wallén', 'Nessrin', 'Lindqvist', 'Lofthag', 'Hansen', 'Lokala', 'Krist', 'Trolle', 'Marit', 'Bengtsson', 'Sjögren', 'Ron', 'singel', 'Mario', 'Malnar', 'KC', 'Taki', 'bergman', 'Johanna', 'Reinfo20mg', 'Lucas', 'Cecila', 'Leyl', 'Saj', 'lös', 'On', 'Stierner', 'Metidazo', 'Hussein', 'Aktiv', 'PatTV', 'Barbro', 'Rahdi', 'Lith', 'Diskuss', 'Sadeer', 'Mahdimg', 'Therése', 'Bettina', 'Pettersson', 'Claes', 'Löfström', 'Hewan', 'Tommaso', 'Padulaont', 'Goshgarian', 'Is', 'Hijlkirurg', 'Maja', 'Sanjiaj', 'Sala', 'Helmi', 'Lip', 'Pacemaker', 'Damoon', 'pat', 'Eugen', 'Cecilila', 'Magda', 'Pateo', 'Hedda', 'Linn', 'Tveit', 'Pettersen', 'Orbitatem', 'TempBond', 'Partner', 'TV', 'Psykodonti', 'Barbara', 'Marek', 'Zoniuk', 'Romeborn', 'Joacim', 'Herculite', 'Sri', 'Mohamed', 'Söker', 'Lilly', 'Buccal', 'Judit', 'Sana', 'Mathilda', 'Noor', 'Ald', 'Un', 'Patff', 'Dor', 'Sally', 'Haux', 'Svalin', 'Sture', 'Cave', 'Rasmus', 'NaCdy', 'Samira', 'Blomqvist', 'Kurbana', 'Trombyict', 'Vaselin', 'MTA', 'Angelusont', 'Sagitta', 'Corsodyldy', 'Erdal', 'Roland', 'Robert', 'Petersför', 'Alveolen', 'Akpen', 'Andergrand', 'Brita', 'Holmström', 'Helena', 'Et', 'Ivoclar', 'Vivaden', 'Put', 'wais', 'Björk', 'Ansi', 'Walsalmn', 'Pink', 'Dystrofia', 'Myoton', 'MH', 'Conny', 'Ingvar', 'Mirvais', 'Mandibla', 'Nahida', 'Patlle', 'Ase', 'Katja', 'Forsberg', 'Ahmad', 'Rel', 'Wisma', 'Ber', 'Angelica', 'Rasmusson', 'Rosann', 'Matilda', 'Kä', 'Örhnell', 'ASP', 'Upp', 'Ehlers', 'DanlosÄK', 'Céline', 'Ni', 'Helfer', 'Pamaro', 'Almehed', 'Egidia', 'HC', 'Tove', 'Hülya', 'Cevik', 'Ara', 'Käkkirurg', 'Erikssontikkliniken', 'TM', 'Oliwer', 'Tsk', 'EnarssonN2O', 'Wan', 'Teresi', 'Asma', 'Ex', 'et', 'Edona', 'Kristian', 'Vestgärd', 'Angela', 'Terapi', 'Sendoline', 'Nr', 'Hassan', 'Alert', 'Enarsson', 'Frans', 'Wigenste', 'Pa', 'Suzan', 'do', 'If', 'Ingalill', 'Inganäs', 'Leon', 'Håkan', 'IngaliÄL', 'Ribbond', 'Revolution', 'Flow', 'Shakar', 'max', 'SOL', 'Fredrick', 'Sender', 'Elnaz', 'Charlotte', 'Heijl', 'Nyman', 'Molander', 'Diagnos', 'Engström', 'IV', 'Stangenberg', 'Lennholm', 'Behosh', 'Malekza', 'Mical', 'Tsega', 'Ay', 'Bertilsson', 'Silver', 'DOota', 'Liselott', 'Löfquist', 'Dahlin', 'Tramadolado', 'Björns', 'Lövendal', 'manuelltan', 'Käkir', 'elsson', 'LEF', 'Lund', 'LÖ', 'Carolina', 'Pattd', 'Preop', 'Marko', 'Ant', 'Lundin', 'Heil', 'Schander', 'Yasin', 'Mros', 'Zahra', 'Salva', 'Göransson', 'Pamensperiod', 'Börresen', 'Fin', 'Bonding', 'Excite', 'Vivapen', 'Lydia', 'Anneli', 'Ale', 'AN', 'Antani', 'Sub', 'Nathan', 'lillebror', 'Josefin', 'Jacqueline', 'Elzer', 'Fråg', 'Gen', 'Solo', 'BK', 'Sin', 'Wid', 'Marigon', 'Wern', 'bor', 'Aurell', 'Fransson', 'KSS', 'Apsav', 'Vita', 'Azar', 'Oscar', 'Carin', 'Wallgren', 'Lot', 'Helén', 'Svensson', 'V2b', 'Mårtensson', 'Helene', 'Amelie', 'Trombyvastform', 'Petersén', 'Ragn', 'Göte', 'Anadagar', 'Dormi', 'Tam', 'Stella', 'Gudnado´ttir', 'SG', 'Flod', 'Nouhan', 'Nourhan', 'mand', 'Sorayasep', 'Odontolog', 'AC', 'Behnoosh', 'Johnson', 'Kimberly', 'Lee', 'Salv', 'Lilian', 'Bråmark', 'Emilio', 'Siv', 'Knarrström', 'Deb', 'Premering', 'Raouf', 'Lagerblad', 'Tallroth', 'Tallrothoka', 'Rangmar', 'Konsta', 'Nobel', 'Essixsken', 'Pat19', 'Fatane', 'Farjami', 'Oldin', 'Zachrisson', 'PASCAL', 'Vighage', 'Mal', 'Dala', 'Dalasin', 'Midazolam', 'Najda', 'Ldy', 'Kul', 'Katarina', 'Sven', 'Edahlarina', 'Görans', 'Strid', 'Friberg', 'ASSA', 'Boden', 'Rönt', 'Prim', 'Nifi', 'Post', 'Rexon', 'Cina', 'Friedy', 'PAC', 'Stig', 'Das', 'Aseel', 'Nashir', 'Sören', 'Dana', 'Rasoul', 'Hel', 'Walsal', 'Vladomir', 'Ivarsson', 'Active', 'Gunnel', 'Xeroden', 'Dentan', 'Joe', 'Rondar', 'Gina', 'Hassel', 'Gustavsson', 'Prolia', 'Web', 'Le', 'Asp', 'Alvar', 'jan', 'Dakinsmg', 'Apelgren', 'PL', 'Picsar', 'Fjelddahl', 'Ass', 'Staveby', 'Fjeldahl', 'svakvak', 'DIagn', 'Cassandra', 'Bakiris', 'Levande', 'Opex', 'Thapper', 'Westberg', 'Leona', 'Malmberg', 'Ghaza', 'Boromand', 'Joneus', 'Privat', 'SU', 'Bilgor', 'Pre', 'Heléna', 'Kasper', 'BU', 'Henning', 'Suleiman', 'IP', 'Rememaby', 'Jävert', 'Hagberg', 'Tand', 'Lindehag', 'Tramadols', 'Alvaro', 'Campos', 'Augustsson', 'Cle', 'Mirandamg', 'bar', 'MSA', 'Xyloc', 'Adrena', 'TDL', 'Gratian', 'Rosca', 'Bou', 'Rahal', 'Mahdi', 'Perköm', 'Anita', 'Isackson', 'Henningsson', 'Lotta', 'Karazisi', 'Synth', 'Mina', 'Mehrpour', 'Ashkoor', 'Hyg', 'Nasrin', 'Kumulin', 'Kå', 'Andri', 'Thorarinsson', 'Synthes', 'Leonardo', 'sep', 'Antony', 'Malsep', 'Nar', 'Ponjevic', 'Panodi', 'Brufen', 'Algina', 'Brateleloche', 'Deltis', 'Rosbecker', 'AnnikaIPLA', 'Lundhal', 'Kronogård', 'Henric', 'Kevin', 'louise', 'Marg', 'Block', 'Stör', 'Forte', 'Hillevi', 'sven', 'Rap', 'Wessam', 'Sanad', 'Kf', 'Ram', 'Szalaj', 'Faride', 'Jesper', 'Besime', 'Syl', 'Osseo', 'Speed', 'Selimovic', 'Trombyli', 'Welander', 'Zakr', 'Ekwall', 'Samuel', 'Bagges', 'Terr', 'Pathak', 'Straumankir', 'Stra', 'Rozita', 'Strauman', 'Ahlberg', 'Herliz', 'Hans', 'Herlitz', 'Kåpen', 'car', 'Petruson', 'Rane', 'Polcit', 'Freegen', 'Cristasnit', 'Lundberg', 'Majly', 'Hermansson', 'LindaTV', 'Ving', 'Bernt', 'Isberg', 'Dak10', 'Ca', 'Oxiner', 'Alvaros', 'Bettfys', 'Åland', 'Diffus', 'lena', 'Ma', 'Patrisia', 'Börgu', 'Grankvist', 'Fu', 'Aspholm', 'Anger', 'Pont', 'Mualem', 'Zelano', 'Tdg', 'Kh', 'Astmaorr', 'Oralt', 'Kem', 'Majvor', 'Goran', 'Mawlood', 'Arnemn', 'Novpara', 'Alv', 'Zen', 'Sheida', 'AST', 'Skallsjö', 'Hak', 'Björlin', 'Vivian', 'Terese', 'Udevalla', 'Cyril', 'Mun', 'Jelena', 'Jokic', 'Ske', 'Harrweg', 'Magnus', 'Gläntan', 'patofer', 'Collis', 'Curve', 'Magnusoka', 'Inf', 'Saga', 'Elgquist', 'XT', 'Ebba', 'ERI', 'Franz', 'Ismajli', 'Hasseus', 'Fjeldal', 'Annerud', 'Anform', 'Marielle', 'Hadil', 'Bondesson', 'Ga', 'Werlenius', 'Tronje', 'Herman', 'Ehle', 'Danin', 'TERAPIPLAN', 'Fris', 'Liby', 'Röntgen', 'mannström', 'JohansonH', 'Lillieström', 'Rodriguezcit', 'Tetofl', 'Evy', 'Luna', 'Meta', 'Ferm', 'Sjögrenby', 'Lindahl', 'Distansia', 'Dingle', 'Benny', 'Ytterby', 'Lucy', 'Jev', 'Patricia150mg', 'Glyde', 'Götberg', 'Kaor', 'Emilsson', 'Hellström', 'Demens', 'Bio', 'Gidelen', 'Nordström', 'PGA', 'Parodoe', 'Damberg', 'Hasséus', 'Hema', 'DIAGNOS', 'OP', 'Amberucveolärutskottetutskottet', 'Kenneth', 'MB', 'ANES', 'Bendamustin', 'Olofe', 'Bas', 'TEPIP', 'KJ', 'Hemlin', 'hy', 'Katarin', 'Krönlein', 'Fix', 'Margit', 'Igelström', 'Gul', 'Regi', 'Lila', 'Regio', 'Melany', 'Elina', 'ssk', 'Felicia', 'Suska', 'Perfalganfal', 'Dagant', 'Tom', 'Vär', 'Matti', 'Koivuniemi', 'Jonika', 'Gjorgjieva', 'Bogren', 'Ir', 'Mörk', 'Savicutan', 'CP', 'Kartous', 'AnnaMaj', 'Molande', 'Löf', 'Duraphat', 'yhn', 'Monika', 'El', 'Aaraj', 'Mes', 'Thorén', 'Dakosomg', 'Trom', 'TEPIPLA', 'rielsson', 'Yngve', 'Submand', 'Telht', 'Chlum', 'Cr', 'Unosson', 'Bartha', 'Patstillfället', 'Ib', 'Rön', 'Pine', 'Hor', 'Metform', 'Humelin', 'Aprov', 'Krönle', 'vondt', 'tan', 'Pas', 'Rehmann', 'Lenberg', 'Salwas', 'Tele', 'Nal0', 'Berg', 'Adnan', 'Maryam', 'Dakinsöp', 'Edta', 'JJK', 'Easybond', 'Dyr', 'Coe', 'soft', 'Kla', 'Ulfmg', 'Kent', 'Sköldborg', 'Bent', 'Neuro', 'Vinita', 'Stemol', 'Theo', 'Adna', 'Jonas', 'Sundberg', 'Pasjiaj', 'Tisse', 'Tisseill', 'NC', 'Ingbritt', 'Ekstedt', 'KristinaHenriksson', 'Thörnqvist', 'Trysberg', 'Estelle', 'Ilona', 'Henriksson', 'Boglar', 'Feket', 'Filip', 'JM', 'Harn', 'Melander', 'Hugosson', 'Jarl', 'Hoschang', 'Ulceros', 'Col', 'Myelo', 'Toner', 'Remica', 'Stenhardt', 'Vaslava', 'Warmark', 'Stenberg', 'Jaan', 'Sv', 'Sam', 'Marne', 'Act', 'Landelius', 'Dagny', 'Blomgren', 'Tobias', 'Fagrell', 'Wol', 'Trombylonoxip', 'Hägglin', 'Fagerberg', 'Mohlin', 'Pent', 'Sahlgrens', 'Tailun', 'Tailu', 'Folicke', 'Astdy', 'Astma', 'Geona', 'Signeden', 'HW', 'Pålsson', 'Exemestan', 'Kalsepos', 'Trimetrop', 'Edvinson', 'Scotch', 'ScotchBond', 'FiltekSupreme', 'Johnny', 'Patienten', 'Saris', 'Jonsson', 'Muhammed', 'Kkir', 'Ahlqwist', 'Margareta', 'BS', 'Haugen', 'Torun', 'Bågenholm', 'Amalgammn', 'Red', 'Bull', 'Edita', 'Hagman', 'Sebastian', 'Dormicum', 'EM', 'Sand', 'Frimannson', 'Ramstedt', 'Artman', 'Winkelhoff', 'Telio', 'BT', 'Ramhult', 'ANA', 'Etchin', 'Mon', 'Karen', 'Daksep', 'JJ', 'Alg', 'Roines', 'Mynutu', 'Nicholas', 'Pär', 'Brhan', 'Brhane', 'Ehrndal', 'Hultgren', 'Hematologen', 'Bend', 'Kalcipos', 'Dystervik', 'Syster', 'Ingemar', 'Ssk', 'Veronica', 'CJ', 'Hullberg', 'Alendrona', 'BRO', 'Kirsi', 'IvanoffÖ', 'Daganteckning', 'Paientdog', 'Durap', 'Plackers', 'Tepe', 'Jih', 'Nord', 'Strömhäll', 'Karl', 'Eric', 'Diklena', 'Dur', 'ZOE', 'PatDen', 'Kommer', '´Ser', 'STV', 'Bankvall', 'Riva', 'Marcus', 'LEA', 'Isik', 'Dagante', 'Manuell', 'ADIM', 'Viva', 'Bett', 'Jeanette', 'Sonya', 'Santos', 'Kia', 'Marcel', 'Putlt', 'Hök', 'Bra', 'Billie', 'Flander', 'Joselin', 'Anesteoka', 'Pascal', 'Furix', 'NaF', 'Ua', 'ell', 'Durario', 'LesnukhinO', 'Nystime', 'Pick', 'Mid1', 'PatF0', 'Margitji', 'Sonifl', 'Ekulf', 'Alterno', 'Nas', 'CLW', 'Nasal', 'CL', 'Zendium', 'Känd', 'Odont', 'Bäckman', 'Tell', 'Ramone', 'Ahlin', 'Gunnila', 'Pentti', 'KE', 'Rev', 'Mild', 'RDA', 'Övl', 'Lill', 'Calciche', 'Paroeime', 'Agnes', 'Wallmark', 'Miranda', 'MI', 'Vallgren', 'Reciproc', 'Rose', 'Kerr', 'Ce', 'Moa', 'Folktand', 'Brican', 'Arthrotec', 'Pan', 'rub', 'Vincent', 'Akhygien', 'Evelina', 'Ernesa', 'Naz', 'Mila', 'Bauer', 'Gunhild', 'Emrén', 'Charlotta', 'Widerström', 'Socialt', 'AM', 'Timote', 'Liljhagen', 'Rödström', 'Triamcinolo', 'Etch', 'Kjellerpri', 'Line', 'Wennerholm', 'Noushin', 'Nordlander', 'Sjukhus', 'Patlien', 'Extraktion', 'Gerd', 'Riise', 'IL', 'Plac', 'Oraqix', 'Mint', 'Svanberg', 'Yasser', 'AliTidi', 'Anthmyr', 'Pihl', 'Törnblom', 'Dridol', 'Opquova', 'Vega', 'Liv', 'Heggen', 'EJ', 'VivaP', 'Adhesreme', 'Bendest', 'Maxis', 'Thomasian', 'Sig', 'Anamnesgiv', 'Yearmn', 'VU', 'TePe', 'Angle', 'isb', 'Karolina', 'Berwald', 'Sanna', 'Årsjö', 'Saif', 'Al', 'Saidy', 'Remfreinet', 'Harward', 'Frondelius', 'Aisomn', 'EvoFlow', 'Maisa', 'Walman', 'Rydén', 'Mircea', 'Albut', 'Aktuellt', 'Christin', 'Midola', 'Flotek', 'Lukas', 'Janerot', 'Datum', 'Ioa', 'Fourgel', 'nilsson', 'Rebecka', 'Mamm', 'Gl', 'Sek', 'Emilia', 'Milding', 'Anela', 'Pentamix', 'Garant', 'ök', 'Puty', 'Almina', 'André', 'Lagarto', 'Apht', 'Gingivit', 'Jendeby', 'Ze', 'Mimmi', 'R1', 'Benhosh', 'Karazi', 'Foss', 'Lorata', 'Bert', 'Endodont', 'Kant', 'Durphat', 'Apodos', 'Nou', 'DOgammn', 'Ketak', 'Ulrikaph', 'Andrèa', 'Dalbergmn', 'Ster', 'PaganderTerapiplan', 'Ellinor', 'Klobetasol', 'Nidal', 'Said', 'Namg', 'DS', 'Easy', 'UKspr', 'Severinsson', 'Yusu', 'Yusuf', 'Re', 'Asaco', 'ind', 'SF', 'Listerine', 'JC', 'Emil', 'Mathias', 'Larborg', 'RA', 'Tb', 'Jörgen', 'Viktor', 'Rim', 'Badawi', 'Flux', 'Dry', 'Mouth', 'Hesam', 'Sowlati', 'Manesh', 'ïdag', 'Kost', 'Rein', 'Glide', 'MG', 'Emine', 'Asperg', 'Torstensson', 'Akutuin10', 'Maud', 'Wirén', 'Heidi', 'Patod', 'Margita', 'Prel', 'Lum', 'Rinse', 'Ad', 'Dyract', 'Sprit', 'Benneth', 'Striver', 'Me', 'Stri', 'Rita', 'Sar', 'XTE', 'Nyst', 'Topden', 'Jashar', 'Dakmn', 'Kav', 'Bro', 'Yearoka', 'Ek', 'Gi', 'KD', 'stensjö', 'Bergman', 'Elvans', 'Sowlat', 'Eivor', 'Bay', 'HjelmFolk', 'Tony', 'charlott', 'Pir', 'Thalin', 'Hun', 'Botox', 'vård', 'Mahvas', 'Mahvash', 'Öv', 'Laila', 'Jansson', 'Medium', 'Kim', 'Lillemor', 'Ekenstråle', 'Esp', 'Romex', 'Kristenson', 'Ved', 'Dig', 'Bamse', 'Tek', 'Löfgren', 'Mazin', 'And', 'Amiira', 'Fernheim', 'Wojciech', 'Gierszewski', 'Mehr', 'Bot', 'Jouni', 'Hassing', 'Neg', 'Valutur', 'Rapide', 'Lokalanestes', 'Riskbedömning', 'Revus', 'Patzin', 'Marina', 'MP', 'Rosita', 'KO', 'AndersTh', 'AnderssonF', 'Gingivits', 'Emelie', 'Sjölin', 'kil', 'Renstan', 'Crohn', 'Billman', 'Michelsqu', 'KauT', 'Valtre', 'Sertralin', 'Jonathan', 'Anton', 'Talal', 'Parwin', 'Holmgren', 'Vuxen', 'Zekiye', 'Helisa', 'Parodmn', 'Pay', 'Fine', 'konstantoka', 'Alveoler', 'Xden', 'Antic', 'Frank', 'Manuelavdelning', 'Gente', 'Lidholms', 'Lidholm', 'TW', 'Annamg', 'Hasan', 'AnamnesS', 'Sarawood', 'Kazmn', 'Slh', 'Appelstrand', 'Emanuel', 'Fanny', 'Bankeryd', 'Rosenlund', 'WallgrenF', 'TS', 'bokar', 'Kvarford', 'Petronella', 'Norman', 'Jimmyod', 'Frankmn', 'Pens', 'Tar', 'Joakim', 'Adhes', 'Sensody', 'Mou', 'Heg', 'FA', 'Jer', 'Stefanos', 'Cheram', 'Ödman', 'Caesar', 'Tolk', 'Dari', 'Svårt', 'Jagers', 'Daniels', 'Narges', 'Easypi', 'Sama', 'Erika', 'Moberg', 'Khan', 'Stomvall', 'Mucocel', 'Ethel', 'Lindholm', 'co', 'Kola', 'Light', 'Hüly', 'Trybom', 'Rydbjer', 'Gustafsson', 'Duraf', 'Pat1', 'Streman', 'Ina', 'Källström', 'Folk', 'Renhat', 'Viola', 'Hälsodekl', 'Gösta', 'Winqvist', 'DOL', 'dagltan', 'Joel', 'Henelund', 'Pauline', 'Mirjam', 'Colgate', 'Ringa', 'Michels', 'Fre', 'Sea', 'Eyvor', 'Olle', 'Aiman', 'Pla', 'Rek', 'Sarvenaz', 'Barwaago', 'Triage', 'Lennartsson', 'Bulimi', 'Kfgam', 'sms', 'Durhat', 'RY', 'Mustafa', 'Karioe', 'Yasmin', 'Vlademir', 'Zarzioovi', 'KarinAn', 'Juk', 'Määtt', 'Xarelto', 'Curettage', 'Vivacit', 'Bulle', 'ProExpert', 'Loka', 'Liza', 'Björndal', 'väninna', 'Guldvingen', 'Solveig', 'Dl', 'Carlgren', 'Tä', 'Nad', 'TI', 'Göthberg', 'Jill', 'Patudi', 'Aya', 'Ayas', 'Ashko', 'grejer', 'Lichen', 'NFR', 'Hiba', 'Charlott', 'Victoria', 'Jasmin', 'Mohammad', 'Annas', 'Munasar', 'SMA', 'Karins', 'Elzbieta', 'Tuggumi', 'mob', 'Albin', 'Glider', 'Gliders', 'Wivi', 'Hygienis', 'Jennie', 'Robinson', 'Af', 'Lisette', 'Andolex1', 'Hjelmgren', 'Anars', 'Irén', 'Pålssons', 'Kristin', 'Kau', 'Lg', 'Amber', 'Päivi', 'Alenfors', 'Elias', 'Zalin', 'Chatrine', 'Klovansky', 'Ellen', 'Jumbo', 'ESPbon', 'Gudrun', 'Pata5', 'Morit', 'Bergman101', 'Nannmark', 'Bucco', 'Attara', 'Bjärne', 'IBLAND', 'Loise', 'Jana', 'BrittSof', 'Peggy', 'Widmans', 'Res', 'WaO', 'Prima', 'Large', 'Soft', 'Junestål', 'Grönqvist', 'JE', 'MGA', 'Shirine', 'Naj', 'Ts', 'Karla', 'Norberg', 'Chi', 'Difluca', 'Amelia', 'Vycr', 'Treiman', 'Capio', 'VC', 'JB', 'Ceceilia', 'Lagersköld', 'Britta', 'Hjördis', 'Toile', 'Trollér', 'Qvist', 'Nicolas', 'lie', 'Sagerbrink', 'Sundbye', 'Charlie', 'Zahran', 'Bahr', 'Valmire', 'Valmir', 'HV', 'Britney', 'Ahlinder', 'Ariana', 'Henrikv', 'Carola', 'Sn', 'WaveOne', 'Math', 'Kamal', 'Mathee', 'Oretad', 'Alveood', 'Leila', 'Hjälpmedel', 'Sjögrens', 'Syndrom', 'in', 'Hjälp', 'Karlssonmn', 'Feras', 'Eid', 'Ria', 'Methotraxa', 'Humira', 'Margaretha', 'Åhman', 'Andreasson', 'Westerhäll', 'gumm', 'Candida', 'Albican', 'Av', 'Xero', 'Melisa', 'Rosa', 'Käkkirug', 'Sakairoun', 'Duote', 'AKL', 'Ljungholm', 'Riitta', 'Hengameh', 'Pirjan', 'Greta', 'Camill', 'Panavia', 'Basilvon', 'Prime', 'NT', 'Sci', 'Samhall', 'Börje', 'Philippa', 'Baumgren', 'Fn', 'Jacob', 'made', 'Bohlin', 'Öl', 'Rochett', 'Levis', 'Karigam', 'Grå', 'Caroline', 'Fridh', 'Hamdi', 'Benyamin', 'SC', 'manuellody', 'Jonatan', 'Kontroll', 'Tavegyl', 'ÖTdl', 'Meijersjö', 'Unique', 'Ilida', 'Thomasiant', 'ESPE', 'Joanna', 'Prote', 'Warlin', 'Prof', 'Anamn1', 'Ove', 'SM', 'Nystimex', 'Kazi', 'Triageod', 'Down', 'Madde', 'Stana', 'Mad', 'Ju', 'Gyn', 'Linnmg', 'Andrei', 'info', 'Christoffer', 'Darin', 'Pierre', 'Tutuml', 'Ridderheim', 'Borcic', 'Pro', 'Ribas', 'Jones', 'Vallmo', 'Manuelgs', 'Citodod', 'Brus', 'Eluiqu', 'CH', 'Patdyl', 'Harrys', 'Awara', 'Oförändrad', 'Angspr', 'line', 'Lilja', 'Annica', 'Adiels', 'Orvar', 'Flex', 'Primreme', 'Rebecca', 'Ceder', 'Birger', 'Molly', 'Blend', 'Ets', 'Nylund', 'Ringhald', 'Nor', 'Stv3A', 'Fru', 'Huntington', 'Dalheimers', 'Polhem', 'Davidsson', 'Cro', 'sahlin', 'Sahlin10', 'Åhrén', 'Akutt', 'Corsdyl', 'Melker', 'Bedömning', 'Leukemi', 'Kai', 'Robin', 'Mirja', 'Kriisa', 'Rung', 'Thunfors', 'Nisse', 'Vogme', 'Vog', 'Wilma', 'Susanna', 'Jonzon', 'Jon', 'US', 'Oföränd', 'ST', 'Peterson', 'Yearole', 'dagllack', 'Sophia', 'Bergbom', 'Belma', 'Marcelle', 'Bertilby', 'Mol', 'Svea', 'Kyl', 'Artrit', 'Nap', 'Cer', 'JacobsenYea', 'Williams', 'Boivie', 'Melaton', 'Warvinge', 'Amani', 'Ragnhild', 'Björklund', 'Warving', 'Zag', 'Zagroz', 'Williamson', 'Ståhl', 'Gull', 'Cleaner', 'Anestesiolarutskottet', 'Hakan', 'Oganezova', 'Bankvalltas', 'Siw', 'Mell', 'Lju', 'Vevskoski', 'Tegret', 'Melvin', 'Fras', 'Paulina', 'Ståhlucona', 'Lampe', 'Collins', 'Co', 'Josef', 'Vass', 'Behep', 'Patisa', 'Cure', 'Tet', 'Cur', 'Färg', 'Mikkelsenmn', 'Utför', 'Rahil', 'Katt', 'konstantmn', 'TIArotepby', 'Kalcip', 'Lola', 'Quenselmn', 'Sose', 'Bjarne', 'Rolf', 'Dub', 'Mediciner', 'Odlogisk', 'Reyes', 'Lindström', 'Eliquep', 'Fatemeh', 'ai', 'Cathrine', 'Nylén', 'Heracill', 'Andreasont', 'Shamso', 'Ginghat', 'Bäck', 'Gullbritt', 'At', 'Chx', 'Pen', 'Pattan', 'Se', 'Carnemar', 'Rosemary', 'Bartfay', 'WarogeAMN', 'Boguslaw', 'Ursula', 'Frost', 'Anamnporges', 'Pradaxa', 'Adrian', 'Esperanza', 'Trimon', 'Cornelia', 'Dagkl', 'Tormn', 'Cy', 'DIAGN', 'Flagerup', 'Herpes', 'Zooster', 'Olofsson', 'Halina', 'Wenneberg', 'Hutchin', 'Bamyl', 'Luite', 'Omar', 'Shewakh', 'Desloratadine', 'Ljung', 'Tilde', 'LÖkr', 'Linnéa', 'Richard', 'Susan', 'Joachim', 'Kicki', 'Perköoöm', 'Annki', 'Nydén', 'Ossner', 'Arnell', 'Kallas', 'Jaredsson', 'Heimdahl', 'Muntorr', 'Ultra', 'Bader', 'Svenne', 'Kimberley', 'Knapman', 'Knapm', 'Evofl', 'Zekrya', 'File', 'Rite', 'EDTAsep', 'Forlax', 'Barmans', 'Norgren', 'Paroeod', 'Magnusson', 'Transpl', 'DIAGNOSER', 'Napro', 'Cucieman', 'Hudläkare', 'Hörfält', 'Denise', 'Wivianne', 'Wågsjöac']

for i in range(len(names)):
    line = names[i]
    names[i] = line.lower()

print(names[:10])

In [ ]:
dataset_antibiotics['Anteckning'][5443]

In [ ]:
c = 0
for n in names:
    remove_rows = remove_name(dataset_antibiotics,n)
    c+= len(remove_rows)
    
print(c)

In [ ]:
dataset_antibiotics['Anteckning'][5443]

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(dataset_antibiotics)
    dataset_antibiotics.to_csv('../data/dataset_no_recipe.csv', index = False)

In [ ]:
dataset_antibiotics['Anteckning'][5443]

# Create Text-to-Text dataset
Some models, such as T5, mT5 and text-to-text models expect the input, label and task to be as text and not as separate features.  
A separate dataset for those models was therefore created

In [ ]:
dataset_antibiotics_seq = dataset_antibiotics.copy(deep=True)
dataset_antibiotics_seq['Anteckning'] = 'Antibiotics classification: ' + dataset_antibiotics_seq['Anteckning'].astype(str)
dataset_antibiotics_seq['Label'] = np.where(dataset_antibiotics_seq['Label'] == 1, 'Positive', 'Negative')

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(dataset_antibiotics_seq)
    dataset_antibiotics_seq.to_csv('../data/seq_dataset.csv', index = False)

In [ ]:
type(dataset_antibiotics_seq['Label'][0])